In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rna_sequence_model_binding_score/transformers/default/1/config.json
/kaggle/input/rna_sequence_model_binding_score/transformers/default/1/merges.txt
/kaggle/input/rna_sequence_model_binding_score/transformers/default/1/training_args.bin
/kaggle/input/rna_sequence_model_binding_score/transformers/default/1/tokenizer.json
/kaggle/input/rna_sequence_model_binding_score/transformers/default/1/vocab.json
/kaggle/input/rna_sequence_model_binding_score/transformers/default/1/tokenizer_config.json
/kaggle/input/rna_sequence_model_binding_score/transformers/default/1/model.safetensors
/kaggle/input/rna_sequence_model_binding_score/transformers/default/1/special_tokens_map.json
/kaggle/input/silico_tokenizer/transformers/default/1/merges.txt
/kaggle/input/silico_tokenizer/transformers/default/1/tokenizer.json
/kaggle/input/silico_tokenizer/transformers/default/1/vocab.json
/kaggle/input/silico_tokenizer/transformers/default/1/tokenizer_config.json
/kaggle/input/silico_tokenizer/tra

In [2]:
!pip install -q streamlit
!pip install datasets==2.14.4
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 70.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 107.0 MB/s eta 0:00:0000:01
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 10.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Found existing installation: datasets 3.5.0
    Uninstalling dataset

In [4]:
%%writefile app.py

import torch

from transformers import AutoTokenizer, GPT2ForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/livingsilico/model/tokenizer")

# gptmodel = GPT2ForSequenceClassification.from_pretrained("/content/drive/MyDrive/livingsilico/model_checkpoint")
# tokenizer.pad_token = tokenizer.eos_token
# model.config.pad_token_id = model.config.eos_token_id
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import sys
sys.path.append("/kaggle/input/rnagenai/transformers/default/1/")

def sampling(out_path,num_samples,start,max_new_tokens,ckpt_path,tokenizer_path,strategy,temperature):
  import sys
  import os
  import pickle
  from contextlib import nullcontext
  import torch
  import tiktoken
  # sys.path.insert(1,'/content/drive/MyDrive/livingsilico/model')
  # export PYTHONPATH='/content/drive/MyDrive/livingsilico/model'
  # sys.path.append('/content/drive/MyDrive/livingsilico/model')

  # os.environ['PYTHONPATH'] = '/content/drive/MyDrive/livingsilico/model'



  from model import GPTConfig, GPT
  from tqdm import tqdm
  import random
  import numpy as np
  from transformers import AutoTokenizer
  from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode
  import argparse
  import itertools
  import random
  import math

  import inspect
  from dataclasses import dataclass

  import torch
  import torch.nn as nn
  from torch.nn import functional as F

  class LayerNorm(nn.Module):
      def __init__(self, ndim, bias):
          super().__init__()
          self.weight = nn.Parameter(torch.ones(ndim))
          self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

      def forward(self, input):
          return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

  class CausalSelfAttention(nn.Module):

      def __init__(self, config):
          super().__init__()
          assert config.n_embd % config.n_head == 0
          # key, query, value projections for all heads, but in a batch
          self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
          # output projection
          self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
          # regularization
          self.attn_dropout = nn.Dropout(config.dropout)
          self.resid_dropout = nn.Dropout(config.dropout)
          self.n_head = config.n_head
          self.n_embd = config.n_embd
          self.dropout = config.dropout
          self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
          if not self.flash:
              print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
              # causal mask to ensure that attention is only applied to the left in the input sequence
              self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                          .view(1, 1, config.block_size, config.block_size))

      def forward(self, x):
          B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

          # calculate query, key, values for all heads in batch and move head forward to be the batch dim
          q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
          k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
          q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
          v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

          # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
          if self.flash:
              # efficient attention using Flash Attention CUDA kernels
              y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
          else:
              # manual implementation of attention
              att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
              att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
              att = F.softmax(att, dim=-1)
              att = self.attn_dropout(att)
              y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
          y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

          # output projection
          y = self.resid_dropout(self.c_proj(y))
          return y

  class MLP(nn.Module):

      def __init__(self, config):
          super().__init__()
          self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
          self.gelu    = nn.GELU()
          self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
          self.dropout = nn.Dropout(config.dropout)

      def forward(self, x):
          x = self.c_fc(x)
          x = self.gelu(x)
          x = self.c_proj(x)
          x = self.dropout(x)
          return x

  class Block(nn.Module):

      def __init__(self, config):
          super().__init__()
          self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
          self.attn = CausalSelfAttention(config)
          self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
          self.mlp = MLP(config)

      def forward(self, x):
          x = x + self.attn(self.ln_1(x))
          x = x + self.mlp(self.ln_2(x))
          return x

  @dataclass
  class GPTConfig:
      block_size: int = 1024
      vocab_size: int = 50304
      n_layer: int = 12
      n_head: int = 12
      n_embd: int = 768
      dropout: float = 0.0
      bias: bool = True

  class GPT(nn.Module):

      def __init__(self, config):
          super().__init__()
          assert config.vocab_size is not None
          assert config.block_size is not None
          self.config = config

          self.transformer = nn.ModuleDict(dict(
              wte = nn.Embedding(config.vocab_size, config.n_embd),
              wpe = nn.Embedding(config.block_size, config.n_embd),
              drop = nn.Dropout(config.dropout),
              h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
              ln_f = LayerNorm(config.n_embd, bias=config.bias),
          ))
          self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
          self.transformer.wte.weight = self.lm_head.weight

          # init all weights
          self.apply(self._init_weights)
          # apply special scaled init to the residual projections, per GPT-2 paper
          for pn, p in self.named_parameters():
              if pn.endswith('c_proj.weight'):
                  torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

          # report number of parameters
          print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

      def get_num_params(self, non_embedding=True):
          """
          Return the number of parameters in the model.
          For non-embedding count (default), the position embeddings get subtracted.
          The token embeddings would too, except due to the parameter sharing these
          params are actually used as weights in the final layer, so we include them.
          """
          n_params = sum(p.numel() for p in self.parameters())
          if non_embedding:
              n_params -= self.transformer.wpe.weight.numel()
          return n_params

      def _init_weights(self, module):
          if isinstance(module, nn.Linear):
              torch.nn.init.kaiming_normal_(module.weight, a=0, mode='fan_in', nonlinearity='relu')
              if module.bias is not None:
                  torch.nn.init.zeros_(module.bias)
          elif isinstance(module, nn.Embedding):
              torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

      def forward(self, idx, targets=None):
          device = idx.device
          b, t = idx.size()
          assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
          pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

          # forward the GPT model itself
          tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
          pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
          x = self.transformer.drop(tok_emb + pos_emb)
          for block in self.transformer.h:
              x = block(x)
          x = self.transformer.ln_f(x)

          if targets is not None:
              # if we are given some desired targets also calculate the loss
              logits = self.lm_head(x)
              loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
          else:
              # inference-time mini-optimization: only forward the lm_head on the very last position
              logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
              loss = None

          return logits, loss

      def crop_block_size(self, block_size):
          # model surgery to decrease the block size if necessary
          # e.g. we may load the GPT2 pretrained model checkpoint (block size 1024)
          # but want to use a smaller block size for some smaller, simpler model
          assert block_size <= self.config.block_size
          self.config.block_size = block_size
          self.transformer.wpe.weight = nn.Parameter(self.transformer.wpe.weight[:block_size])
          for block in self.transformer.h:
              if hasattr(block.attn, 'bias'):
                  block.attn.bias = block.attn.bias[:,:,:block_size,:block_size]

      @classmethod
      def from_pretrained(cls, model_type, override_args=None):
          assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
          override_args = override_args or {} # default to empty dict
          # only dropout can be overridden see more notes below
          assert all(k == 'dropout' for k in override_args)
          from transformers import GPT2LMHeadModel
          print("loading weights from pretrained gpt: %s" % model_type)

          # n_layer, n_head and n_embd are determined from model_type
          config_args = {
              'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
              'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
              'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
              'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
          }[model_type]
          print("forcing vocab_size=50257, block_size=1024, bias=True")
          config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
          config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
          config_args['bias'] = True # always True for GPT model checkpoints
          # we can override the dropout rate, if desired
          if 'dropout' in override_args:
              print(f"overriding dropout rate to {override_args['dropout']}")
              config_args['dropout'] = override_args['dropout']
          # create a from-scratch initialized minGPT model
          config = GPTConfig(**config_args)
          model = GPT(config)
          sd = model.state_dict()
          sd_keys = sd.keys()
          sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

          # init a huggingface/transformers model
          model_hf = GPT2LMHeadModel.from_pretrained(model_type)
          sd_hf = model_hf.state_dict()

          # copy while ensuring all of the parameters are aligned and match in names and shapes
          sd_keys_hf = sd_hf.keys()
          sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
          sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
          transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
          # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
          # this means that we have to transpose these weights when we import them
          assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
          for k in sd_keys_hf:
              if any(k.endswith(w) for w in transposed):
                  # special treatment for the Conv1D weights we need to transpose
                  assert sd_hf[k].shape[::-1] == sd[k].shape
                  with torch.no_grad():
                      sd[k].copy_(sd_hf[k].t())
              else:
                  # vanilla copy over the other parameters
                  assert sd_hf[k].shape == sd[k].shape
                  with torch.no_grad():
                      sd[k].copy_(sd_hf[k])

          return model

      def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
          # start with all of the candidate parameters
          param_dict = {pn: p for pn, p in self.named_parameters()}
          # filter out those that do not require grad
          param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
          # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
          # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
          decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
          nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
          optim_groups = [
              {'params': decay_params, 'weight_decay': weight_decay},
              {'params': nodecay_params, 'weight_decay': 0.0}
          ]
          num_decay_params = sum(p.numel() for p in decay_params)
          num_nodecay_params = sum(p.numel() for p in nodecay_params)
          print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
          print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
          # Create AdamW optimizer and use the fused version if it is available
          fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
          use_fused = fused_available and device_type == 'cuda'
          extra_args = dict(fused=True) if use_fused else dict()
          optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
          print(f"using fused AdamW: {use_fused}")

          return optimizer

      def estimate_mfu(self, fwdbwd_per_iter, dt):
          """ estimate model flops utilization (MFU) in units of A100 bfloat16 peak FLOPS """
          # first estimate the number of flops we do per iteration.
          # see PaLM paper Appendix B as ref: https://arxiv.org/abs/2204.02311
          N = self.get_num_params()
          cfg = self.config
          L, H, Q, T = cfg.n_layer, cfg.n_head, cfg.n_embd//cfg.n_head, cfg.block_size
          flops_per_token = 6*N + 12*L*H*Q*T
          flops_per_fwdbwd = flops_per_token * T
          flops_per_iter = flops_per_fwdbwd * fwdbwd_per_iter
          # express our flops throughput as ratio of A100 bfloat16 peak flops
          flops_achieved = flops_per_iter * (1.0/dt) # per second
          flops_promised = 312e12 # A100 GPU bfloat16 peak flops is 312 TFLOPS
          mfu = flops_achieved / flops_promised
          return mfu

      @torch.no_grad()
      def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None, strategy='sampling', beam_size=3, eos_token_id=0, repetition_penalty=1.0):
          assert strategy in ['greedy_search', 'sampling', 'top_k', 'beam_search']

          batch_size = idx.size(0)
          if strategy == 'beam_search':
              # Initialize beams
              beam_seqs = [idx.clone() for _ in range(beam_size)]
              beam_scores = torch.zeros((batch_size, beam_size), device=idx.device)
              completed_seqs = []

              for _ in range(max_new_tokens):
                  all_candidates = []
                  for i in range(beam_size):
                      idx_cond = beam_seqs[i] if beam_seqs[i].size(1) <= self.config.block_size else beam_seqs[i][:, -self.config.block_size:]
                      logits, _ = self(idx_cond)
                      logits = logits[:, -1, :] / temperature
                      if repetition_penalty != 1.0:
                          for j in range(idx_cond.size(1)):
                              logits[:, idx_cond[:, j]] /= repetition_penalty
                      probs = F.log_softmax(logits, dim=-1)
                      scores, indices = torch.topk(probs, beam_size, dim=-1)

                      for j in range(beam_size):
                          candidate_seq = torch.cat([beam_seqs[i], indices[:, j:j+1]], dim=1)
                          candidate_score = beam_scores[:, i] + scores[:, j]
                          if indices[0, j] == eos_token_id:
                              completed_seqs.append((candidate_score, candidate_seq))
                          else:
                              all_candidates.append((candidate_score, candidate_seq))

                  # add random noise when sorting beacause that generated sequences of beam_search remain unchanged if they have the same prefix
                  all_candidates.sort(key=lambda x: x[0].mean().item() + torch.rand(1).item() * 5e-1, reverse=True)

                  beam_seqs = [all_candidates[i][1] for i in range(min(beam_size, len(all_candidates)))]
                  beam_scores = torch.stack([all_candidates[i][0] for i in range(min(beam_size, len(all_candidates)))], dim=1)
                  if len(completed_seqs) >= beam_size:
                      break

              if not completed_seqs:
                  completed_seqs = [(beam_scores[:, i], beam_seqs[i]) for i in range(beam_size)]

              completed_seqs.sort(key=lambda x: x[0].mean().item(), reverse=True)
              return completed_seqs[0][1]


          else:
              for _ in range(max_new_tokens):
                  idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
                  logits, _ = self(idx_cond)
                  logits = logits[:, -1, :] / temperature

                  if repetition_penalty != 1.0:
                      for i in range(idx.size(0)):
                          for j in range(idx.size(1)):
                              logits[i, idx[i, j]] /= repetition_penalty

                  if strategy == 'greedy_search':
                      idx_next = torch.argmax(logits, dim=-1, keepdim=True)

                  elif strategy == 'sampling':
                      probs = F.softmax(logits, dim=-1)
                      idx_next = torch.multinomial(probs, num_samples=1)

                  elif strategy == 'top_k':
                      if top_k is not None:
                          logits, indices = torch.topk(logits, min(top_k, logits.size(-1)))
                          probs = F.softmax(logits, dim=-1)
                          idx_next = torch.multinomial(probs, num_samples=1)
                          idx_next = torch.gather(indices, dim=-1, index=idx_next)

                  if idx_next == eos_token_id:
                      break
                  idx = torch.cat((idx, idx_next), dim=1)

          return idx if idx[0][0] != eos_token_id else idx[:, 1:]

  # num_samples = num_samples
  # max_new_tokens = max_new_tokens
  # ckpt_path = ckpt_path
  # tokenizer_path = tokenizer_path
  if start==None: start= "<|endoftext|>"
  # -----------------------------------------------------------------------------
  parser = argparse.ArgumentParser()
  # parser.add_argument("--init_from", type=str, default="resume", help="Directory of raw data & output files")
  init_from="resume"
  # parser.add_argument("--out_path", type=str, required=True)

  # parser.add_argument("--num_samples", type=int, required=False, default=num_samples)

  # parser.add_argument("--max_new_tokens", type=int, required=True, default=max_new_tokens,help="number of tokens generated in each sample")

  # parser.add_argument("--strategy",type=str, required=False,default='top_k',help="should be in ['greedy_search', 'sampling', 'top_k', 'beam_search']")
 
  # parser.add_argument("--beam_size",type=int, required=False,default=3,help="beam size for beam search")
  beam_size="3"
  # parser.add_argument("--temperature",type=float, required=False,default=1.0,help="1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions")
  
  # parser.add_argument("--top_k",type=int, required=False,default=20,help="retain only the top_k most likely tokens, clamp others to have 0 probability")
  top_k=20  
  # parser.add_argument("--ckpt_path",type=str, required=True,default = ckpt_path,help="path to a checkpoint/model")

  # parser.add_argument("--tokenizer_path",type=str, required=True,default= tokenizer_path, help="path to a tokenizer directory")

  # parser.add_argument("--start",type=str, required=False,default=start)

  # parser.add_argument("--repetition_penalty",type=float, required=False,default=1.0)
  repetition_penalty=1.0
  # parser.add_argument("--shuffle_token", action='store_true', help="Enable shuffling of tokens before decoding")
  shuffle_token=False
  # parser.add_argument("--fasta", action='store_true', default=True, help="Enable writing output in FASTA format")
  fasta=True
  # args = parser.parse_args()

  # print(args)

  # init_from = args.init_from

  # out_path = args.out_path

  # num_samples = args.num_samples

  # max_new_tokens = args.max_new_tokens

  # strategy = args.strategy
  assert strategy in ['greedy_search', 'sampling', 'top_k', 'beam_search']
  # beam_size = args.beam_size
  # temperature = args.temperature
  # top_k = args.top_k

  # ckpt_path = args.ckpt_path
  # tokenizer_path = args.tokenizer_path
  # start = args.start

  # repetition_penalty = args.repetition_penalty
  # fasta = args.fasta


  # -----------------------------------------------------------------------------
  seed = random.randint(1,6666)
  device = 'cuda'
  dtype = 'float32'
  # dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
  compile = False # use PyTorch 2.0 to compile the model to be faster

  # -----------------------------------------------------------------------------
  # Load tokenizer & ckpt_path
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
  # -----------------------------------------------------------------------------

  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
  torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
  device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
  ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
  ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

  # model
  if init_from == 'resume':
      # init from a model saved in a specific directory
      checkpoint = torch.load(ckpt_path, map_location=device)
      gptconf = GPTConfig(**checkpoint['model_args'])
      model = GPT(gptconf)
      state_dict = checkpoint['model']
      unwanted_prefix = '_orig_mod.'
      for k,v in list(state_dict.items()):
          if k.startswith(unwanted_prefix):
              state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
      model.load_state_dict(state_dict)
  elif init_from.startswith('gpt2'):
      # init from a given GPT-2 model
      model = GPT.from_pretrained(init_from, dict(dropout=0.0))

  model.eval()
  model.to(device)
  if compile:
      model = torch.compile(model) # requires PyTorch 2.0 (optional)

  # look for the meta pickle in case it is available in the dataset folder
  load_meta = False
  encode = tokenizer.encode
  decode = tokenizer.decode

  fasta_out_path = os.path.splitext(out_path)[0] + ".fasta" if fasta else None

  result =[]

  if strategy in["sampling", "top_k"]:
      start_ids = encode("".join(start))
      x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])


      # with open(out_path, 'a') as f:
      #     with open(fasta_out_path, 'a') if fasta else nullcontext() as fasta_f:
      #         with torch.no_grad():
      #             with ctx:
      #                 for k in tqdm(range(num_samples), desc="Generating samples"):
      #                     token_sequence = model.generate(x, max_new_tokens, strategy=strategy, temperature=temperature, top_k=top_k, repetition_penalty=repetition_penalty)[0].tolist()

      #                     # Shuffle tokens if --shuffle_token is specified
      #                     if args.shuffle_token:
      #                         random.shuffle(token_sequence)

      #                     y = decode(token_sequence).replace(' ', '')
      #                     # y = decode(token_sequence).replace('\n', '').replace(' ', '') + '\n'
      #                     f.write(y)
      #                     f.flush()


      #                     if fasta:
      #                         fasta_entry = f">sample_{k}\n{y.replace(' ', '')}\n"
      #                         fasta_f.write(fasta_entry.strip() + '\n')
      #                         fasta_f.flush()
      with torch.no_grad():
          with ctx:
              for k in tqdm(range(num_samples), desc="Generating samples"):
                  token_sequence = model.generate(x, max_new_tokens, strategy=strategy, temperature=temperature, top_k=top_k, repetition_penalty=repetition_penalty)[0].tolist()

                          # Shuffle tokens if --shuffle_token is specified
                  if shuffle_token: #args.shuffle_token:
                      random.shuffle(token_sequence)

                  y = decode(token_sequence).replace(' ', '')
                  y = decode(token_sequence).replace('\n', '').replace(' ', '') + '\n'

                  result.append(y)

  elif strategy in ["beam_search", "greedy_search"]:


      # with open(out_path, 'a') as f:
      #     with open(fasta_out_path, 'a') if fasta else nullcontext() as fasta_f:
      #         with torch.no_grad():
      #             with ctx:
      #                 start = '<|endoftext|>'
      #                 start_ids = encode(start)
      #                 x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

      #                 token_sequence = model.generate(x, max_new_tokens, strategy=strategy, temperature=temperature, top_k=top_k, repetition_penalty=repetition_penalty, beam_size=beam_size)[0].tolist()

      #                 y = decode(token_sequence).replace(' ', '')
      #                 f.write(y)
      #                 f.flush()


      #                 if fasta:
      #                     fasta_entry = f">sample_{k}\n{y.replace(' ', '')}\n"
      #                     fasta_f.write(fasta_entry.strip() + '\n')
      #                     fasta_f.flush()

            # with open(out_path, 'a') as f:
      #     with open(fasta_out_path, 'a') if fasta else nullcontext() as fasta_f:

      with torch.no_grad():
          with ctx:
              start = '<|endoftext|>'
              start_ids = encode(start)
              x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

              token_sequence = model.generate(x, max_new_tokens, strategy=strategy, temperature=temperature, top_k=top_k, repetition_penalty=repetition_penalty, beam_size=beam_size)[0].tolist()

              y = decode(token_sequence).replace(' ', '')
              result.append(y)
  return result

def predict_score(sequence):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/silico_tokenizer/transformers/default/1")

    model = GPT2ForSequenceClassification.from_pretrained("/kaggle/input/rna_sequence_model_binding_score/transformers/default/1").to(device)
    tokenizer.pad_token = tokenizer.eos_token

    model.config.pad_token_id = model.config.eos_token_id
    inputs = tokenizer(sequence, return_tensors="pt", padding=True, truncation=True)
    # Move inputs to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs).logits

    # Convert scaled prediction back to original range
    RMSD_prediction = outputs.item()
    # original_prediction = scaler.inverse_transform([[scaled_prediction]])[0][0]

    return {
        "RMSD_prediction": RMSD_prediction
        # "original_prediction": original_prediction
    }

# # Example prediction
# example_sequence = "AGCUGAGAGAGUACAUCAGGUGGGAAGAAGCUGCAAGGAAUUUGCUGGGUCUCAUAGAAGCAAAGGAGAACAGAAACCACCAGCCACCUCAACCCAAGGCCCUGGGCAAUCAGCAGCCUUCGUGGGAUUCAGAGGAUAGCAGCAACUUCAAAGAUUGGUAGACUCUCUGCUCCAGGUUCUCAACGUGAAGGAAGGAACCCCC"
# print(f"Example sequence: {example_sequence[:50]}...")
# print(f"Prediction: {predict_score(example_sequence)}")
import sys
sys.path.append("/kaggle/input/rnagenai/transformers/default/1/")
# from model import GPTConfig, GPT

max_new_tokens = 256

out_path ="/kaggle/working/outputsequence/output.txt"
ckpt_path = "/kaggle/input/rnagenai/transformers/default/1/model_updated.pt"
tokenizer_path = "/kaggle/input/silico_tokenizer/transformers/default/1/"

# num_samples = 5
# start ='AGC'
# strategy="top_k"
# temperature=1.0

import streamlit as st
if "visibility" not in st.session_state:
    st.session_state.visibility = "visible"
    st.session_state.disabled = False

col1, col2 = st.columns(2)

generated=None


with col1:
    strategy = st.radio(
        "Select Optimization Strategy 👇",
        # key="strategy",
        options=['top_k','greedy_search', 'sampling', 'beam_search']
    )
    num_samples= st.text_input(
        "Sample Size",2
    )
    start = st.text_input(
        "Starting protein","<|endoftext|>"
    )
    temperature = st.text_input(
        "Temperature",1.0
    )
    
    if col1.button("Let's generate", use_container_width=True):
      # if strategy or num_samples or temperature:
      #     st.write("Please select strategy")
      # else:
        
      # if start: start = "<|endoftext|>"
          
      generated = sampling(out_path,int(num_samples),start,max_new_tokens,ckpt_path,tokenizer_path,strategy,float(temperature))
      predicted =[ predict_score(seq).get("RMSD_prediction") for seq in generated]
      # st.write(pd.DataFrame(
      #   {
      #       "Generated Sequences": generated,
      #       "Predicted RMSD Score": predicted,
      #   }))

      
import pandas as pd          
if generated:
    st.write(pd.DataFrame(
        {
            "Generated Sequences": generated,
            "Predicted RMSD Score": predicted,
        }))


# sampling(out_path,num_samples,start,max_new_tokens,ckpt_path,tokenizer_path,strategy,temperature)

with col2:
    text_input = st.text_input(
        "Enter RNA Sequence 👇"
        # label_visibility=st.session_state.visibility,
        # disabled=st.session_state.disabled,
        # placeholder=st.session_state.placeholder,
    )
    if col2.button("Let's predict", use_container_width=True):


      if text_input:
        st.write("Predicted RMSD Score: ", predict_score(text_input).get("RMSD_prediction"))
      else:
        st.write("Please enter a sequence")
          
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


Overwriting app.py


In [5]:
!curl https://loca.lt/mytunnelpassword

35.192.126.186

In [6]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 2s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸npm notice
npm notice New major version of npm available! 10.8.2 -> 11.3.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.3.0
npm notice To update run: npm install -g npm@11.3.0
npm notice
⠸

In [ ]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦your url is: https://ripe-deer-punch.loca.lt


In [ ]:
# AGCUGAGAGAGUACAUCAGGUGGGAAGAAGCUGCAAGGAAUUUGCUGGGUCUCAUAGAAGCAAAGGAGAACAGAAACCACCAGCCACCUCAACCCAAGGCCCUGGGCAAUCAGCAGCCUUCGUGGGAUUCAGAGGAUAGCAGCAACUUCAAAGAUUGGUAGACUCUCUGCUCCAGGUUCUCAACGUGAAGGAAGGAACCCCC

In [ ]:

# def sampling(out_path,num_samples,start,max_new_tokens,ckpt_path,tokenizer_path,strategy,temperature):
#   import sys
#   import os
#   import pickle
#   from contextlib import nullcontext
#   import torch
#   import tiktoken
#   # sys.path.insert(1,'/content/drive/MyDrive/livingsilico/model')
#   # export PYTHONPATH='/content/drive/MyDrive/livingsilico/model'
#   # sys.path.append('/content/drive/MyDrive/livingsilico/model')

#   # os.environ['PYTHONPATH'] = '/content/drive/MyDrive/livingsilico/model'



#   from model import GPTConfig, GPT
#   from tqdm import tqdm
#   import random
#   import numpy as np
#   from transformers import AutoTokenizer
#   from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode
#   import argparse
#   import itertools
#   import random
#   import math

#   import inspect
#   from dataclasses import dataclass

#   import torch
#   import torch.nn as nn
#   from torch.nn import functional as F

#   class LayerNorm(nn.Module):
#       def __init__(self, ndim, bias):
#           super().__init__()
#           self.weight = nn.Parameter(torch.ones(ndim))
#           self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

#       def forward(self, input):
#           return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

#   class CausalSelfAttention(nn.Module):

#       def __init__(self, config):
#           super().__init__()
#           assert config.n_embd % config.n_head == 0
#           # key, query, value projections for all heads, but in a batch
#           self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
#           # output projection
#           self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
#           # regularization
#           self.attn_dropout = nn.Dropout(config.dropout)
#           self.resid_dropout = nn.Dropout(config.dropout)
#           self.n_head = config.n_head
#           self.n_embd = config.n_embd
#           self.dropout = config.dropout
#           self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
#           if not self.flash:
#               print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
#               # causal mask to ensure that attention is only applied to the left in the input sequence
#               self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
#                                           .view(1, 1, config.block_size, config.block_size))

#       def forward(self, x):
#           B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

#           # calculate query, key, values for all heads in batch and move head forward to be the batch dim
#           q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
#           k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
#           q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
#           v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

#           # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
#           if self.flash:
#               # efficient attention using Flash Attention CUDA kernels
#               y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
#           else:
#               # manual implementation of attention
#               att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
#               att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
#               att = F.softmax(att, dim=-1)
#               att = self.attn_dropout(att)
#               y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
#           y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

#           # output projection
#           y = self.resid_dropout(self.c_proj(y))
#           return y

#   class MLP(nn.Module):

#       def __init__(self, config):
#           super().__init__()
#           self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
#           self.gelu    = nn.GELU()
#           self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
#           self.dropout = nn.Dropout(config.dropout)

#       def forward(self, x):
#           x = self.c_fc(x)
#           x = self.gelu(x)
#           x = self.c_proj(x)
#           x = self.dropout(x)
#           return x

#   class Block(nn.Module):

#       def __init__(self, config):
#           super().__init__()
#           self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
#           self.attn = CausalSelfAttention(config)
#           self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
#           self.mlp = MLP(config)

#       def forward(self, x):
#           x = x + self.attn(self.ln_1(x))
#           x = x + self.mlp(self.ln_2(x))
#           return x

#   @dataclass
#   class GPTConfig:
#       block_size: int = 1024
#       vocab_size: int = 50304
#       n_layer: int = 12
#       n_head: int = 12
#       n_embd: int = 768
#       dropout: float = 0.0
#       bias: bool = True

#   class GPT(nn.Module):

#       def __init__(self, config):
#           super().__init__()
#           assert config.vocab_size is not None
#           assert config.block_size is not None
#           self.config = config

#           self.transformer = nn.ModuleDict(dict(
#               wte = nn.Embedding(config.vocab_size, config.n_embd),
#               wpe = nn.Embedding(config.block_size, config.n_embd),
#               drop = nn.Dropout(config.dropout),
#               h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
#               ln_f = LayerNorm(config.n_embd, bias=config.bias),
#           ))
#           self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
#           self.transformer.wte.weight = self.lm_head.weight

#           # init all weights
#           self.apply(self._init_weights)
#           # apply special scaled init to the residual projections, per GPT-2 paper
#           for pn, p in self.named_parameters():
#               if pn.endswith('c_proj.weight'):
#                   torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

#           # report number of parameters
#           print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

#       def get_num_params(self, non_embedding=True):
#           """
#           Return the number of parameters in the model.
#           For non-embedding count (default), the position embeddings get subtracted.
#           The token embeddings would too, except due to the parameter sharing these
#           params are actually used as weights in the final layer, so we include them.
#           """
#           n_params = sum(p.numel() for p in self.parameters())
#           if non_embedding:
#               n_params -= self.transformer.wpe.weight.numel()
#           return n_params

#       def _init_weights(self, module):
#           if isinstance(module, nn.Linear):
#               torch.nn.init.kaiming_normal_(module.weight, a=0, mode='fan_in', nonlinearity='relu')
#               if module.bias is not None:
#                   torch.nn.init.zeros_(module.bias)
#           elif isinstance(module, nn.Embedding):
#               torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

#       def forward(self, idx, targets=None):
#           device = idx.device
#           b, t = idx.size()
#           assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
#           pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

#           # forward the GPT model itself
#           tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
#           pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
#           x = self.transformer.drop(tok_emb + pos_emb)
#           for block in self.transformer.h:
#               x = block(x)
#           x = self.transformer.ln_f(x)

#           if targets is not None:
#               # if we are given some desired targets also calculate the loss
#               logits = self.lm_head(x)
#               loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
#           else:
#               # inference-time mini-optimization: only forward the lm_head on the very last position
#               logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
#               loss = None

#           return logits, loss

#       def crop_block_size(self, block_size):
#           # model surgery to decrease the block size if necessary
#           # e.g. we may load the GPT2 pretrained model checkpoint (block size 1024)
#           # but want to use a smaller block size for some smaller, simpler model
#           assert block_size <= self.config.block_size
#           self.config.block_size = block_size
#           self.transformer.wpe.weight = nn.Parameter(self.transformer.wpe.weight[:block_size])
#           for block in self.transformer.h:
#               if hasattr(block.attn, 'bias'):
#                   block.attn.bias = block.attn.bias[:,:,:block_size,:block_size]

#       @classmethod
#       def from_pretrained(cls, model_type, override_args=None):
#           assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
#           override_args = override_args or {} # default to empty dict
#           # only dropout can be overridden see more notes below
#           assert all(k == 'dropout' for k in override_args)
#           from transformers import GPT2LMHeadModel
#           print("loading weights from pretrained gpt: %s" % model_type)

#           # n_layer, n_head and n_embd are determined from model_type
#           config_args = {
#               'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
#               'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
#               'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
#               'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
#           }[model_type]
#           print("forcing vocab_size=50257, block_size=1024, bias=True")
#           config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
#           config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
#           config_args['bias'] = True # always True for GPT model checkpoints
#           # we can override the dropout rate, if desired
#           if 'dropout' in override_args:
#               print(f"overriding dropout rate to {override_args['dropout']}")
#               config_args['dropout'] = override_args['dropout']
#           # create a from-scratch initialized minGPT model
#           config = GPTConfig(**config_args)
#           model = GPT(config)
#           sd = model.state_dict()
#           sd_keys = sd.keys()
#           sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

#           # init a huggingface/transformers model
#           model_hf = GPT2LMHeadModel.from_pretrained(model_type)
#           sd_hf = model_hf.state_dict()

#           # copy while ensuring all of the parameters are aligned and match in names and shapes
#           sd_keys_hf = sd_hf.keys()
#           sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
#           sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
#           transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
#           # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
#           # this means that we have to transpose these weights when we import them
#           assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
#           for k in sd_keys_hf:
#               if any(k.endswith(w) for w in transposed):
#                   # special treatment for the Conv1D weights we need to transpose
#                   assert sd_hf[k].shape[::-1] == sd[k].shape
#                   with torch.no_grad():
#                       sd[k].copy_(sd_hf[k].t())
#               else:
#                   # vanilla copy over the other parameters
#                   assert sd_hf[k].shape == sd[k].shape
#                   with torch.no_grad():
#                       sd[k].copy_(sd_hf[k])

#           return model

#       def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
#           # start with all of the candidate parameters
#           param_dict = {pn: p for pn, p in self.named_parameters()}
#           # filter out those that do not require grad
#           param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
#           # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
#           # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
#           decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
#           nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
#           optim_groups = [
#               {'params': decay_params, 'weight_decay': weight_decay},
#               {'params': nodecay_params, 'weight_decay': 0.0}
#           ]
#           num_decay_params = sum(p.numel() for p in decay_params)
#           num_nodecay_params = sum(p.numel() for p in nodecay_params)
#           print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
#           print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
#           # Create AdamW optimizer and use the fused version if it is available
#           fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
#           use_fused = fused_available and device_type == 'cuda'
#           extra_args = dict(fused=True) if use_fused else dict()
#           optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
#           print(f"using fused AdamW: {use_fused}")

#           return optimizer

#       def estimate_mfu(self, fwdbwd_per_iter, dt):
#           """ estimate model flops utilization (MFU) in units of A100 bfloat16 peak FLOPS """
#           # first estimate the number of flops we do per iteration.
#           # see PaLM paper Appendix B as ref: https://arxiv.org/abs/2204.02311
#           N = self.get_num_params()
#           cfg = self.config
#           L, H, Q, T = cfg.n_layer, cfg.n_head, cfg.n_embd//cfg.n_head, cfg.block_size
#           flops_per_token = 6*N + 12*L*H*Q*T
#           flops_per_fwdbwd = flops_per_token * T
#           flops_per_iter = flops_per_fwdbwd * fwdbwd_per_iter
#           # express our flops throughput as ratio of A100 bfloat16 peak flops
#           flops_achieved = flops_per_iter * (1.0/dt) # per second
#           flops_promised = 312e12 # A100 GPU bfloat16 peak flops is 312 TFLOPS
#           mfu = flops_achieved / flops_promised
#           return mfu

#       @torch.no_grad()
#       def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None, strategy='sampling', beam_size=3, eos_token_id=0, repetition_penalty=1.0):
#           assert strategy in ['greedy_search', 'sampling', 'top_k', 'beam_search']

#           batch_size = idx.size(0)
#           if strategy == 'beam_search':
#               # Initialize beams
#               beam_seqs = [idx.clone() for _ in range(beam_size)]
#               beam_scores = torch.zeros((batch_size, beam_size), device=idx.device)
#               completed_seqs = []

#               for _ in range(max_new_tokens):
#                   all_candidates = []
#                   for i in range(beam_size):
#                       idx_cond = beam_seqs[i] if beam_seqs[i].size(1) <= self.config.block_size else beam_seqs[i][:, -self.config.block_size:]
#                       logits, _ = self(idx_cond)
#                       logits = logits[:, -1, :] / temperature
#                       if repetition_penalty != 1.0:
#                           for j in range(idx_cond.size(1)):
#                               logits[:, idx_cond[:, j]] /= repetition_penalty
#                       probs = F.log_softmax(logits, dim=-1)
#                       scores, indices = torch.topk(probs, beam_size, dim=-1)

#                       for j in range(beam_size):
#                           candidate_seq = torch.cat([beam_seqs[i], indices[:, j:j+1]], dim=1)
#                           candidate_score = beam_scores[:, i] + scores[:, j]
#                           if indices[0, j] == eos_token_id:
#                               completed_seqs.append((candidate_score, candidate_seq))
#                           else:
#                               all_candidates.append((candidate_score, candidate_seq))

#                   # add random noise when sorting beacause that generated sequences of beam_search remain unchanged if they have the same prefix
#                   all_candidates.sort(key=lambda x: x[0].mean().item() + torch.rand(1).item() * 5e-1, reverse=True)

#                   beam_seqs = [all_candidates[i][1] for i in range(min(beam_size, len(all_candidates)))]
#                   beam_scores = torch.stack([all_candidates[i][0] for i in range(min(beam_size, len(all_candidates)))], dim=1)
#                   if len(completed_seqs) >= beam_size:
#                       break

#               if not completed_seqs:
#                   completed_seqs = [(beam_scores[:, i], beam_seqs[i]) for i in range(beam_size)]

#               completed_seqs.sort(key=lambda x: x[0].mean().item(), reverse=True)
#               return completed_seqs[0][1]


#           else:
#               for _ in range(max_new_tokens):
#                   idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
#                   logits, _ = self(idx_cond)
#                   logits = logits[:, -1, :] / temperature

#                   if repetition_penalty != 1.0:
#                       for i in range(idx.size(0)):
#                           for j in range(idx.size(1)):
#                               logits[i, idx[i, j]] /= repetition_penalty

#                   if strategy == 'greedy_search':
#                       idx_next = torch.argmax(logits, dim=-1, keepdim=True)

#                   elif strategy == 'sampling':
#                       probs = F.softmax(logits, dim=-1)
#                       idx_next = torch.multinomial(probs, num_samples=1)

#                   elif strategy == 'top_k':
#                       if top_k is not None:
#                           logits, indices = torch.topk(logits, min(top_k, logits.size(-1)))
#                           probs = F.softmax(logits, dim=-1)
#                           idx_next = torch.multinomial(probs, num_samples=1)
#                           idx_next = torch.gather(indices, dim=-1, index=idx_next)

#                   if idx_next == eos_token_id:
#                       break
#                   idx = torch.cat((idx, idx_next), dim=1)

#           return idx if idx[0][0] != eos_token_id else idx[:, 1:]

#   # num_samples = num_samples
#   # max_new_tokens = max_new_tokens
#   # ckpt_path = ckpt_path
#   # tokenizer_path = tokenizer_path
#   if start==None: start= "<|endoftext|>"
#   # -----------------------------------------------------------------------------
#   parser = argparse.ArgumentParser()
#   # parser.add_argument("--init_from", type=str, default="resume", help="Directory of raw data & output files")
#   init_from="resume"
#   # parser.add_argument("--out_path", type=str, required=True)

#   # parser.add_argument("--num_samples", type=int, required=False, default=num_samples)

#   # parser.add_argument("--max_new_tokens", type=int, required=True, default=max_new_tokens,help="number of tokens generated in each sample")

#   # parser.add_argument("--strategy",type=str, required=False,default='top_k',help="should be in ['greedy_search', 'sampling', 'top_k', 'beam_search']")
 
#   # parser.add_argument("--beam_size",type=int, required=False,default=3,help="beam size for beam search")
#   beam_size="3"
#   # parser.add_argument("--temperature",type=float, required=False,default=1.0,help="1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions")
  
#   # parser.add_argument("--top_k",type=int, required=False,default=20,help="retain only the top_k most likely tokens, clamp others to have 0 probability")
#   top_k=20  
#   # parser.add_argument("--ckpt_path",type=str, required=True,default = ckpt_path,help="path to a checkpoint/model")

#   # parser.add_argument("--tokenizer_path",type=str, required=True,default= tokenizer_path, help="path to a tokenizer directory")

#   # parser.add_argument("--start",type=str, required=False,default=start)

#   # parser.add_argument("--repetition_penalty",type=float, required=False,default=1.0)
#   repetition_penalty=1.0
#   # parser.add_argument("--shuffle_token", action='store_true', help="Enable shuffling of tokens before decoding")
#   shuffle_token=False
#   # parser.add_argument("--fasta", action='store_true', default=True, help="Enable writing output in FASTA format")
#   fasta=True
#   # args = parser.parse_args()

#   # print(args)

#   # init_from = args.init_from

#   # out_path = args.out_path

#   # num_samples = args.num_samples

#   # max_new_tokens = args.max_new_tokens

#   # strategy = args.strategy
#   assert strategy in ['greedy_search', 'sampling', 'top_k', 'beam_search']
#   # beam_size = args.beam_size
#   # temperature = args.temperature
#   # top_k = args.top_k

#   # ckpt_path = args.ckpt_path
#   # tokenizer_path = args.tokenizer_path
#   # start = args.start

#   # repetition_penalty = args.repetition_penalty
#   # fasta = args.fasta


#   # -----------------------------------------------------------------------------
#   seed = random.randint(1,6666)
#   device = 'cuda'
#   dtype = 'float32'
#   # dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
#   compile = False # use PyTorch 2.0 to compile the model to be faster

#   # -----------------------------------------------------------------------------
#   # Load tokenizer & ckpt_path
#   tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
#   # -----------------------------------------------------------------------------

#   torch.manual_seed(seed)
#   torch.cuda.manual_seed(seed)
#   torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
#   torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
#   device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
#   ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
#   ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

#   # model
#   if init_from == 'resume':
#       # init from a model saved in a specific directory
#       checkpoint = torch.load(ckpt_path, map_location=device)
#       gptconf = GPTConfig(**checkpoint['model_args'])
#       model = GPT(gptconf)
#       state_dict = checkpoint['model']
#       unwanted_prefix = '_orig_mod.'
#       for k,v in list(state_dict.items()):
#           if k.startswith(unwanted_prefix):
#               state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
#       model.load_state_dict(state_dict)
#   elif init_from.startswith('gpt2'):
#       # init from a given GPT-2 model
#       model = GPT.from_pretrained(init_from, dict(dropout=0.0))

#   model.eval()
#   model.to(device)
#   if compile:
#       model = torch.compile(model) # requires PyTorch 2.0 (optional)

#   # look for the meta pickle in case it is available in the dataset folder
#   load_meta = False
#   encode = tokenizer.encode
#   decode = tokenizer.decode

#   fasta_out_path = os.path.splitext(out_path)[0] + ".fasta" if fasta else None

#   result =[]

#   if strategy in["sampling", "top_k"]:
#       start_ids = encode("".join(start))
#       x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])


#       # with open(out_path, 'a') as f:
#       #     with open(fasta_out_path, 'a') if fasta else nullcontext() as fasta_f:
#       #         with torch.no_grad():
#       #             with ctx:
#       #                 for k in tqdm(range(num_samples), desc="Generating samples"):
#       #                     token_sequence = model.generate(x, max_new_tokens, strategy=strategy, temperature=temperature, top_k=top_k, repetition_penalty=repetition_penalty)[0].tolist()

#       #                     # Shuffle tokens if --shuffle_token is specified
#       #                     if args.shuffle_token:
#       #                         random.shuffle(token_sequence)

#       #                     y = decode(token_sequence).replace(' ', '')
#       #                     # y = decode(token_sequence).replace('\n', '').replace(' ', '') + '\n'
#       #                     f.write(y)
#       #                     f.flush()


#       #                     if fasta:
#       #                         fasta_entry = f">sample_{k}\n{y.replace(' ', '')}\n"
#       #                         fasta_f.write(fasta_entry.strip() + '\n')
#       #                         fasta_f.flush()
#       with torch.no_grad():
#           with ctx:
#               for k in tqdm(range(num_samples), desc="Generating samples"):
#                   token_sequence = model.generate(x, max_new_tokens, strategy=strategy, temperature=temperature, top_k=top_k, repetition_penalty=repetition_penalty)[0].tolist()

#                           # Shuffle tokens if --shuffle_token is specified
#                   if shuffle_token: #args.shuffle_token:
#                       random.shuffle(token_sequence)

#                   y = decode(token_sequence).replace(' ', '')
#                   y = decode(token_sequence).replace('\n', '').replace(' ', '') + '\n'

#                   result.append(y)

#   elif strategy in ["beam_search", "greedy_search"]:


#       # with open(out_path, 'a') as f:
#       #     with open(fasta_out_path, 'a') if fasta else nullcontext() as fasta_f:
#       #         with torch.no_grad():
#       #             with ctx:
#       #                 start = '<|endoftext|>'
#       #                 start_ids = encode(start)
#       #                 x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

#       #                 token_sequence = model.generate(x, max_new_tokens, strategy=strategy, temperature=temperature, top_k=top_k, repetition_penalty=repetition_penalty, beam_size=beam_size)[0].tolist()

#       #                 y = decode(token_sequence).replace(' ', '')
#       #                 f.write(y)
#       #                 f.flush()


#       #                 if fasta:
#       #                     fasta_entry = f">sample_{k}\n{y.replace(' ', '')}\n"
#       #                     fasta_f.write(fasta_entry.strip() + '\n')
#       #                     fasta_f.flush()

#             # with open(out_path, 'a') as f:
#       #     with open(fasta_out_path, 'a') if fasta else nullcontext() as fasta_f:

#       with torch.no_grad():
#           with ctx:
#               start = '<|endoftext|>'
#               start_ids = encode(start)
#               x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

#               token_sequence = model.generate(x, max_new_tokens, strategy=strategy, temperature=temperature, top_k=top_k, repetition_penalty=repetition_penalty, beam_size=beam_size)[0].tolist()

#               y = decode(token_sequence).replace(' ', '')
#               result.append(y)
#   return result

In [ ]:

# import sys
# sys.path.append("/kaggle/input/rnagenai/transformers/default/1/")
# # from model import GPTConfig, GPT
# start ='AGC'
# max_new_tokens = 256
# num_samples = 5
# out_path ="/kaggle/working/outputsequence/output.txt"
# ckpt_path = "/kaggle/input/rnagenai/transformers/default/1/model_updated.pt"
# tokenizer_path = "/kaggle/input/silico_tokenizer/transformers/default/1/"
# strategy="top_k"
# temperature=1.0
# sampling(out_path,num_samples,start,max_new_tokens,ckpt_path,tokenizer_path,strategy,temperature)
